In [23]:
import os, sys
currentdir = os.path.dirname(os.path.abspath(""))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

In [24]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\data")

In [25]:
import joe_thornton as jt
import conor_garland as cg

For each action recorded, the data is structured by frequency and by the impact of the action $a_i$ for all $i \in \{1,2,3\}$ which represent goal, primary assists, and secondary assists respectively.

For each of these types I have constructed 3 matrices $M_i$ that respresent these various actions. Essentially $M_i = \begin{bmatrix} (a_{freq},a_{type}) \; | \; a_{freq} \in \mathbb{N}, a_{sig} \in \{0,...,4\} \end{bmatrix}$ where the variable $a_{sig}$ represents whether the goal was a significant ($a_{sig}=1$), leveling ($a_{sig}=2$), winning ($a_{sig}=3$), clinching point ($a_{sig}=4$), or none of the above ($a_{sig}=0$). Each row of this matrix represents a game, and each game has 13 tuples  that correspond to the relative score of the given player's team.

I plan on rewriting and restructuring this data to eliminate the waste in memory for having a bunch of $(0,0)$ tuples populating a mostly empty matrix.

In [26]:
def getSignificantResults(matrix):
    # this filters out games without recorded actions
    reduced_matrix = [r for r in matrix if sum([j for i,j in r]) != 0]
    sig_obs = [[0]*13 for i in range(0,4)]

    # iterate over the reduced matrix
    for game in reduced_matrix:
        game_transpose = list(zip(*game))

        for sig in range(1,5):
            for i in range(0,13):
                if sig == game_transpose[1][i]:
                    sig_obs[sig-1][i] += game_transpose[0][i]

    return [sum(sig) for sig in zip(*sig_obs)]

In [27]:
# define a series of helper functions to display our results
getResults = lambda M: [sum(game) for game in zip(*[[i for i,j in obs] for obs in M])]
table      = lambda cum_list: "".join(["%3d"]*len(cum_list)) % tuple(cum_list)

In [28]:
def outputResults(name,player):
    cum_sig_goals = getSignificantResults(player.goal_matrix)
    cum_goals = getResults(player.goal_matrix)

    cum_sig_pri_assists = getSignificantResults(player.primary_assist_matrix)
    cum_pri_assists = getResults(player.primary_assist_matrix)

    cum_sig_sec_assists = getSignificantResults(player.secondary_assist_matrix)
    cum_sec_assists = getResults(player.secondary_assist_matrix)

    total_goals   = sum(cum_goals)
    total_assists = sum(cum_sec_assists) + sum(cum_pri_assists)
    total_points  = total_goals + total_assists
    confirmation  = [total_goals,total_assists,total_points]

    total_sig_goals   = sum(cum_sig_goals)
    total_sig_assists = sum(cum_sig_sec_assists) + sum(cum_sig_pri_assists)
    total_sig_points  = total_sig_goals + total_sig_assists
    sig_confirmation  = [total_sig_goals,total_sig_assists,total_sig_points]

    print("\n%-25s%35s" % (name,"points relative to opponent"))
    print("%25s %s" % (" ",table(range(-6,7))))

    print("\n%-25s %s" % ("total goals:",table(cum_goals)))
    print("%-25s %s" % ("total primary assists:",table(cum_pri_assists)))
    print("%-25s %s" % ("total secondary assists:",table(cum_sec_assists)))

    print("\n%-25s %s" % ("sig. goals:",table(cum_sig_goals)))
    print("%-25s %s" % ("sig. primary assists:",table(cum_sig_pri_assists)))
    print("%-25s %s\n" % ("sig. secondary assists:",table(cum_sig_sec_assists)))

    ratios = [sig_confirmation[i]/confirmation[i] for i in range(0,3)]
    print("ratios: ["+', '.join(['%.2f']*3) % tuple(ratios) +"]\n")

In [29]:
outputResults("conor garland",cg)


conor garland                    points relative to opponent
                           -6 -5 -4 -3 -2 -1  0  1  2  3  4  5  6

total goals:                0  0  0  0  2  1  4  0  1  1  0  0  0
total primary assists:      0  0  0  0  6  1  3  4  0  0  1  0  0
total secondary assists:    0  0  0  0  0  0  1  0  0  0  0  0  0

sig. goals:                 0  0  0  0  2  1  4  0  1  1  0  0  0
sig. primary assists:       0  0  0  0  6  1  3  4  0  0  0  0  0
sig. secondary assists:     0  0  0  0  0  0  1  0  0  0  0  0  0

ratios: [1.00, 0.94, 0.96]



In [30]:
outputResults("joe thornton",jt)


joe thornton                     points relative to opponent
                           -6 -5 -4 -3 -2 -1  0  1  2  3  4  5  6

total goals:                0  0  1  0  0  2  0  1  3  0  0  0  0
total primary assists:      0  0  0  0  2  4  5  5  1  1  0  0  0
total secondary assists:    0  0  0  1  3  4  9  6  2  0  0  0  0

sig. goals:                 0  0  0  0  0  2  0  1  0  0  0  0  0
sig. primary assists:       0  0  0  0  2  4  5  5  0  0  0  0  0
sig. secondary assists:     0  0  0  1  3  4  9  6  1  0  0  0  0

ratios: [0.43, 0.93, 0.86]

